In [1]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
%load_ext autoreload
%autoreload 2

# Scraping

First, we scrap the people and Member council tables.

In [2]:
scrap = Scraper()
df_person = scrap.get('Person')
df_member_council = scrap.get('MemberCouncil')

GET: https://ws.parlament.ch/odata.svc/Person?$top=1000&$filter=Language%20eq%20'FR'&$skip=0
GET: https://ws.parlament.ch/odata.svc/Person?$top=1000&$filter=Language%20eq%20'FR'&$skip=1000
GET: https://ws.parlament.ch/odata.svc/Person?$top=1000&$filter=Language%20eq%20'FR'&$skip=2000
GET: https://ws.parlament.ch/odata.svc/Person?$top=1000&$filter=Language%20eq%20'FR'&$skip=3000
GET: https://ws.parlament.ch/odata.svc/Person?$top=1000&$filter=Language%20eq%20'FR'&$skip=4000
[OK] table Person correctly scraped, df.shape =  3525 as expected
GET: https://ws.parlament.ch/odata.svc/MemberCouncil?$top=1000&$filter=Language%20eq%20'FR'&$skip=0
GET: https://ws.parlament.ch/odata.svc/MemberCouncil?$top=1000&$filter=Language%20eq%20'FR'&$skip=1000
GET: https://ws.parlament.ch/odata.svc/MemberCouncil?$top=1000&$filter=Language%20eq%20'FR'&$skip=2000
GET: https://ws.parlament.ch/odata.svc/MemberCouncil?$top=1000&$filter=Language%20eq%20'FR'&$skip=3000
GET: https://ws.parlament.ch/odata.svc/MemberCou

Now, we check the shape of both DF.

In [3]:
print("Length person: ", df_person.shape)
print("Length member council: ", df_member_council.shape)

Length person:  (3525, 21)
Length member council:  (3514, 43)


As we can see, the DF personn is bigger than the DF Member Council. Therefore, we will get the IDs of the persons that are unique in Person.

In [4]:
# IDS That are only in person
id_unique_person = list(set(df_person['ID']) - set(df_member_council['ID']))
print(id_unique_person)

# Create DF of the unique persons
df_unique_person = df_person.loc[df_person['ID'].isin(id_unique_person)]
df_unique_person

['4133', '4043', '832', '830', '1309', '4010', '3991', '3990', '4211', '831', '4127']


,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,MilitaryRank,...,Modified,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
726,1948-03-25T00:00:00,None,Mariangela,f,830,FR,Wallimann-Bornatico,None,None,None,...,2015-05-17T21:18:19.387,I,None,Wallimann-Bornatico,None,830,Grisons,Roveredo,115,lic. iur.
727,1941-10-02T00:00:00,2016-07-17T00:00:00,Achille,m,831,FR,Casanova,None,None,None,...,2016-07-18T15:25:10.473,I,None,Casanova,2581,831,None,None,None,None
728,1947-11-16T00:00:00,None,Hanna,f,832,FR,Muralt Müller,None,None,None,...,2015-05-17T21:18:19.387,D,None,Muralt Müller,2582,832,None,None,None,None
886,1944-01-01T00:00:00,None,Oswald,m,1309,FR,Sigg,None,None,None,...,2015-05-17T21:18:19.387,D,None,Sigg,None,1309,Zurich,Zurich,None,None
3340,1948-06-05T00:00:00,None,Christoph,m,3990,FR,Lanz,2,marié(e),None,...,2015-05-17T21:18:19.387,D,None,Lanz,None,3990,None,None,6,Dr. iur.
3341,1961-08-07T00:00:00,None,Thomas,m,3991,FR,Helbling,None,None,None,...,2016-05-24T13:59:16.603,D,None,Helbling,3111,3991,None,None,112,lic. iur. / LL. M.
3346,1968-11-17T00:00:00,None,André,m,4010,FR,Simonazzi,None,None,None,...,2015-05-17T21:18:19.387,F,None,Simonazzi,2736,4010,Valais,Monthey,None,None
3361,1927-06-06T00:00:00,2012-01-28T00:00:00,Jean-Marc,m,4043,FR,Sauvant,None,None,None,...,2015-05-17T21:18:19.387,F,None,Sauvant,None,4043,None,None,None,None
3443,1964-01-19T00:00:00,None,Philippe,m,4127,FR,Schwab,None,None,None,...,2015-05-17T21:18:19.387,F,None,Schwab,None,4127,Berne,Bienne,273,Master of Public Administration
3448,1964-10-05T00:00:00,None,Martina,f,4133,FR,Buol,None,None,None,...,2015-05-17T21:18:19.387,D,None,Buol,None,4133,Fribourg,Fribourg,6,Dr. iur.


We checked on Wikipedia as well as on http://parlament.ch who are these persons:

- Secretary General of the Federal Assembly:
 - Jean-Marc Sauvant: from 1981-1992
 - Mariangela Wallimann-Bornatico:  from 1999 to 2008
 - Christoph Lanz: from 2008 to 2013
 - Philippe Schwab: from 2013 to Now
 
 
- Vice-Chancelor of Switzerland:
 - Achille Casanova: from 1981 to 2005
 - Oswald Sigg: from 2005 to 2009
 - Hanna Muralt Müller: before 2005
 - Thomas Helbling: from 2008 to 2016
 - André Simonazzi: from 2009 to Now
 - Jörg de Bernardi: from 2016 to Now


- Deputy Secretary General and Secretary of the Council of States:
 - Martina Buol: Inbound
 
These guys didn't have any vote. So, we can simply remove them. 

Now, let's check if other Vice-Chancelor are in the Member Council.

In [5]:
df_person.columns

Index(['DateOfBirth', 'DateOfDeath', 'FirstName', 'GenderAsString', 'ID',
       'Language', 'LastName', 'MaritalStatus', 'MaritalStatusText',
       'MilitaryRank', 'MilitaryRankText', 'Modified', 'NativeLanguage',
       'NumberOfChildren', 'OfficialName', 'PersonIdCode', 'PersonNumber',
       'PlaceOfBirthCanton', 'PlaceOfBirthCity', 'Title', 'TitleText'],
      dtype='object')

In [6]:
# Show the columns in member_council
df_member_council.columns

Index(['Active', 'AdditionalActivity', 'AdditionalMandate',
       'BirthPlace_Canton', 'BirthPlace_City', 'Canton', 'CantonAbbreviation',
       'CantonName', 'Citizenship', 'Council', 'CouncilAbbreviation',
       'CouncilName', 'DateElection', 'DateJoining', 'DateLeaving', 'DateOath',
       'DateOfBirth', 'DateOfDeath', 'DateResignation', 'FirstName',
       'GenderAsString', 'ID', 'IdPredecessor', 'Language', 'LastName',
       'Mandates', 'MaritalStatus', 'MaritalStatusText', 'MilitaryRank',
       'MilitaryRankText', 'Modified', 'NumberOfChildren', 'OfficialName',
       'ParlGroupAbbreviation', 'ParlGroupFunction', 'ParlGroupFunctionText',
       'ParlGroupName', 'ParlGroupNumber', 'Party', 'PartyAbbreviation',
       'PartyName', 'PersonIdCode', 'PersonNumber'],
      dtype='object')

In [7]:
# Take the example of Corina Casanova.
df_member_council[df_member_council['LastName']=='Casanova']

,Active,AdditionalActivity,AdditionalMandate,BirthPlace_Canton,BirthPlace_City,Canton,CantonAbbreviation,CantonName,Citizenship,Council,...,ParlGroupAbbreviation,ParlGroupFunction,ParlGroupFunctionText,ParlGroupName,ParlGroupNumber,Party,PartyAbbreviation,PartyName,PersonIdCode,PersonNumber
883,false,None,None,None,Ilanz,18,GR,Grisons,"Ruschein (GR),Vrin (GR)",98,...,None,None,None,None,None,14,PDC,Parti démocrate-chrétien suisse,2731,1310


In [8]:
# Let's check her Value for Council Name
df_member_council[df_member_council['LastName']=='Casanova']['CouncilName']

883    None
Name: CouncilName, dtype: object

We see here that she's in the member council table. But we also see that her function is "None". Therefore, let's check the unique values for `CouncilName`.

In [9]:
df_member_council['CouncilName'].unique()

array(['Conseil national', 'Conseil des Etats', 'Conseil fédéral', None], dtype=object)

In [10]:
# Extract the None for the CouncilName
df_member_no_council = df_member_council[df_member_council['CouncilName'].isnull()]
df_member_no_council[['LastName', 'FirstName']]

,LastName,FirstName
724,Huber-Hotz,Annemarie
873,Couchepin,François
883,Casanova,Corina
3354,Buser,Walter
3513,Thurnherr,Walter


So, these people are either Chancelor, Vice-Chancelor or from the Secretary General of Switzerland. Therefore, we can remove them from both the DF. 

In [11]:
idx_remove_council = df_member_no_council.index
print(idx_remove_council)

idx_remove_person_1 = df_unique_person.index
print(idx_remove_person_1)
idx_remove_person_2 = df_person[df_person['ID'].isin(list(df_member_no_council['ID']))].index
print(idx_remove_person_2)

idx_remove_person = idx_remove_person_1.union(idx_remove_person_2)
print(idx_remove_person)

Int64Index([724, 873, 883, 3354, 3513], dtype='int64')
Int64Index([726, 727, 728, 886, 3340, 3341, 3346, 3361, 3443, 3448, 3524], dtype='int64')
Int64Index([724, 876, 887, 3362, 3523], dtype='int64')
Int64Index([ 724,  726,  727,  728,  876,  886,  887, 3340, 3341, 3346, 3361,
            3362, 3443, 3448, 3523, 3524],
           dtype='int64')


In [12]:
# No we remove them
df_person_clean = df_person.drop(idx_remove_person)
print('Size person: ', df_person_clean.shape)

df_member_council_clean = df_member_council.drop(idx_remove_council)
print('Size member council: ', df_member_council_clean.shape)

Size person:  (3509, 21)
Size member council:  (3509, 43)


In [13]:
idx = 1297
df_person_clean[df_person_clean.index == idx]['LastName']

1297    Carlin
Name: LastName, dtype: object

In [14]:
df_member_council_clean[df_member_council_clean.index == idx]['LastName']

1297    Carteret
Name: LastName, dtype: object

In [15]:
df_people = df_member_council_clean.merge(df_person_clean, on='ID', suffixes=('_Council', '_Person'))

In [16]:
df_people.shape

(3509, 63)

In [17]:
df_people.head()

,Active,AdditionalActivity,AdditionalMandate,BirthPlace_Canton,BirthPlace_City,Canton,CantonAbbreviation,CantonName,Citizenship,Council,...,Modified_Person,NativeLanguage,NumberOfChildren_Person,OfficialName_Person,PersonIdCode_Person,PersonNumber_Person,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
0,false,Réducteur TSV de 1971 à 1983,"Prés. Féd. Romande des Sociolistes ch., Prés. ...",None,Pompaples,22,VD,Vaud,"Sullens (VD),Lutry (VD)",1,...,2015-05-17T21:18:19.387,F,None,Aguet,2200,1,Vaud,Pompaples,None,None
1,false,None,None,None,None,1,ZH,Zurich,"Kreuzlingen (TG),Fällanden (ZH)",1,...,2015-05-17T21:18:19.387,D,None,Allenspach,2002,2,None,None,None,None
2,false,Zentralpräsident Schweiz. Skliverband 1985 bis...,None,None,Hasle,3,LU,Lucerne,Hasle (LU),1,...,2015-05-17T21:18:19.387,D,5,Aregger,2004,6,Lucerne,Hasle,9,dipl. Bauing. HTL
3,false,None,None,None,None,2,BE,Berne,Tavannes (BE),1,...,2015-05-17T21:18:19.387,F,None,Aubry Geneviève,2005,7,None,None,None,None
4,false,None,None,None,None,2,BE,Berne,"Siselen (BE),Richterswil (ZH)",1,...,2015-05-17T21:18:19.387,D,None,Bär,2008,8,None,None,None,None


In [18]:
columns_both = list(df_person_clean.columns) + list(df_member_council_clean.columns)
columns_people = list(df_people.columns)

In [19]:
both_no_people = list(set(columns_both)-set(columns_people))
people_no_both = list(set(columns_people)-set(columns_both))

In [20]:
for i in both_no_people:
    print('Columns for \'%s\' equals: '%i, df_people[i+'_Council'].equals(df_people[i+'_Person']))

Columns for 'MilitaryRank' equals:  True
Columns for 'MaritalStatusText' equals:  True
Columns for 'FirstName' equals:  True
Columns for 'NumberOfChildren' equals:  True
Columns for 'GenderAsString' equals:  True
Columns for 'LastName' equals:  True
Columns for 'PersonIdCode' equals:  True
Columns for 'OfficialName' equals:  True
Columns for 'Modified' equals:  True
Columns for 'MaritalStatus' equals:  True
Columns for 'DateOfBirth' equals:  True
Columns for 'PersonNumber' equals:  True
Columns for 'MilitaryRankText' equals:  True
Columns for 'DateOfDeath' equals:  True
Columns for 'Language' equals:  True


All columns are the same. So, we can remove one of them.

In [21]:
for i in both_no_people:
    df_people = df_people.drop(i+'_Council', axis=1)
    df_people = df_people.rename(columns={i+'_Person':i})

In [22]:
# display all pandas columns

pd.set_option('display.max_columns', 100)

In [23]:
df_people.head()

,Active,AdditionalActivity,AdditionalMandate,BirthPlace_Canton,BirthPlace_City,Canton,CantonAbbreviation,CantonName,Citizenship,Council,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunction,ParlGroupFunctionText,ParlGroupName,ParlGroupNumber,Party,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,Language,LastName,MaritalStatus,MaritalStatusText,MilitaryRank,MilitaryRankText,Modified,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText
0,false,Réducteur TSV de 1971 à 1983,"Prés. Féd. Romande des Sociolistes ch., Prés. ...",None,Pompaples,22,VD,Vaud,"Sullens (VD),Lutry (VD)",1,CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1,None,"Conseiller communal de 1965 à 1982, Conseiller...",None,None,None,None,None,12,PSS,Parti socialiste suisse,1938-03-02T00:00:00,None,Pierre,m,FR,Aguet,2,marié(e),5,Fourrier,2015-05-17T21:18:19.387,F,None,Aguet,2200,1,Vaud,Pompaples,None,None
1,false,None,None,None,None,1,ZH,Zurich,"Kreuzlingen (TG),Fällanden (ZH)",1,CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,2,None,None,None,None,None,None,None,15,PLR,PLR.Les Libéraux-Radicaux,1928-02-22T00:00:00,None,Heinz,m,FR,Allenspach,None,None,None,None,2015-05-17T21:18:19.387,D,None,Allenspach,2002,2,None,None,None,None
2,false,Zentralpräsident Schweiz. Skliverband 1985 bis...,None,None,Hasle,3,LU,Lucerne,Hasle (LU),1,CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,6,None,Grossrat 1967 bis 1983 (Präsident 1977),None,None,None,None,None,15,PLR,PLR.Les Libéraux-Radicaux,1931-01-27T00:00:00,None,Manfred,m,FR,Aregger,2,marié(e),7,Adjudant sous-officier,2015-05-17T21:18:19.387,D,5,Aregger,2004,6,Lucerne,Hasle,9,dipl. Bauing. HTL
3,false,None,None,None,None,2,BE,Berne,Tavannes (BE),1,CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,7,None,None,None,None,None,None,None,None,None,None,1928-03-04T00:00:00,None,Geneviève,f,FR,Aubry,None,None,None,None,2015-05-17T21:18:19.387,F,None,Aubry Geneviève,2005,7,None,None,None,None
4,false,None,None,None,None,2,BE,Berne,"Siselen (BE),Richterswil (ZH)",1,CN,Conseil national,1987-11-30T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,8,None,None,None,None,None,None,None,None,None,None,1947-12-01T00:00:00,None,Rosmarie,f,FR,Bär,None,None,None,None,2015-05-17T21:18:19.387,D,None,Bär,2008,8,None,None,None,None


In [24]:
# Remove the following columns:
col_to_remove = ['Canton', 'Council', 'ParlGroupFunction', 'ParlGroupNumber', 'Party', 'MaritalStatus', 'MilitaryRank', 'Title', 'BirthPlace_Canton', 'BirthPlace_City', 'Language', 'Modified']
df_people_clean = df_people.drop(col_to_remove, axis=1)

In [25]:
df_people_clean.shape

(3509, 36)

In [26]:
df_people_clean.head()

,Active,AdditionalActivity,AdditionalMandate,CantonAbbreviation,CantonName,Citizenship,CouncilAbbreviation,CouncilName,DateElection,DateJoining,DateLeaving,DateOath,DateResignation,ID,IdPredecessor,Mandates,ParlGroupAbbreviation,ParlGroupFunctionText,ParlGroupName,PartyAbbreviation,PartyName,DateOfBirth,DateOfDeath,FirstName,GenderAsString,LastName,MaritalStatusText,MilitaryRankText,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,TitleText
0,false,Réducteur TSV de 1971 à 1983,"Prés. Féd. Romande des Sociolistes ch., Prés. ...",VD,Vaud,"Sullens (VD),Lutry (VD)",CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1,None,"Conseiller communal de 1965 à 1982, Conseiller...",None,None,None,PSS,Parti socialiste suisse,1938-03-02T00:00:00,None,Pierre,m,Aguet,marié(e),Fourrier,F,None,Aguet,2200,1,Vaud,Pompaples,None
1,false,None,None,ZH,Zurich,"Kreuzlingen (TG),Fällanden (ZH)",CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,2,None,None,None,None,None,PLR,PLR.Les Libéraux-Radicaux,1928-02-22T00:00:00,None,Heinz,m,Allenspach,None,None,D,None,Allenspach,2002,2,None,None,None
2,false,Zentralpräsident Schweiz. Skliverband 1985 bis...,None,LU,Lucerne,Hasle (LU),CN,Conseil national,1995-12-04T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,1995-12-04T00:00:00,1999-12-05T00:00:00,6,None,Grossrat 1967 bis 1983 (Präsident 1977),None,None,None,PLR,PLR.Les Libéraux-Radicaux,1931-01-27T00:00:00,None,Manfred,m,Aregger,marié(e),Adjudant sous-officier,D,5,Aregger,2004,6,Lucerne,Hasle,dipl. Bauing. HTL
3,false,None,None,BE,Berne,Tavannes (BE),CN,Conseil national,1979-11-26T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,1979-11-26T00:00:00,1995-12-03T00:00:00,7,None,None,None,None,None,None,None,1928-03-04T00:00:00,None,Geneviève,f,Aubry,None,None,F,None,Aubry Geneviève,2005,7,None,None,None
4,false,None,None,BE,Berne,"Siselen (BE),Richterswil (ZH)",CN,Conseil national,1987-11-30T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,1987-11-30T00:00:00,1995-12-03T00:00:00,8,None,None,None,None,None,None,None,1947-12-01T00:00:00,None,Rosmarie,f,Bär,None,None,D,None,Bär,2008,8,None,None,None


In [27]:
df_people_clean[df_people_clean['Active'] == 'true'].shape

(253, 36)

There's the good number of active people. (200 National Council, 46 State Council and 7 Federal Council). Let's save it!

In [28]:
df_people_clean.to_csv('data/people.csv', encoding='utf-8', index=False)